In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: False


# Data Preprocessing

In [3]:
import os
os.chdir('..')

df = pd.read_csv('./hourly02-ithaca/hourly02-NY_Ithaca_13_E.csv', header = 0, index_col = 0)

In [5]:
Date = pd.to_datetime(df.UTC_DATE, format='%Y%m%d', errors='coerce')
+ pd.to_timedelta(df.UTC_TIME//100, unit = 'hours')
df['Time'] = Date

In [6]:
df.columns

Index(['WBANNO', 'UTC_DATE', 'UTC_TIME', 'LST_DATE', 'LST_TIME', 'CRX_VN',
       'LONGITUDE', 'LATITUDE', 'T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_FLAG', 'SOLARAD_MAX', 'SOLARAD_MAX_FLAG',
       'SOLARAD_MIN', 'SOLARAD_MIN_FLAG', 'SUR_TEMP_TYPE', 'SUR_TEMP',
       'SUR_TEMP_FLAG', 'SUR_TEMP_MAX', 'SUR_TEMP_MAX_FLAG', 'SUR_TEMP_MIN',
       'SUR_TEMP_MIN_FLAG', 'RH_HR_AVG', 'RH_HR_AVG_FLAG', 'SOIL_MOISTURE_5',
       'SOIL_MOISTURE_10', 'SOIL_MOISTURE_20', 'SOIL_MOISTURE_50',
       'SOIL_MOISTURE_100', 'SOIL_TEMP_5', 'SOIL_TEMP_10', 'SOIL_TEMP_20',
       'SOIL_TEMP_50', 'SOIL_TEMP_100', 'Time'],
      dtype='object')

In [7]:
data = df[['T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_MAX',
       'SOLARAD_MIN', 'SUR_TEMP',
           'SUR_TEMP_MAX', 'SUR_TEMP_MIN', 'RH_HR_AVG']]

In [8]:
data.index = df['Time']

In [9]:
# check for N/A
data.min()

T_CALC         -30.8
T_HR_AVG       -29.3
T_MAX          -28.4
T_MIN          -30.9
P_CALC           0.0
SOLARAD          0.0
SOLARAD_MAX      0.0
SOLARAD_MIN      0.0
SUR_TEMP       -35.8
SUR_TEMP_MAX   -61.0
SUR_TEMP_MIN   -36.0
RH_HR_AVG        0.0
dtype: float64

In [10]:
data

,T_CALC,T_HR_AVG,T_MAX,T_MIN,P_CALC,SOLARAD,SOLARAD_MAX,SOLARAD_MIN,SUR_TEMP,SUR_TEMP_MAX,SUR_TEMP_MIN,RH_HR_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,8.8,NaN,NaN,0.0
2004-10-27,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6.7,NaN,NaN,0.0
2004-10-28,7.8,7.6,8.0,7.3,0.0,0.0,NaN,NaN,6.1,NaN,NaN,0.0
2004-10-28,6.5,7.0,7.8,6.5,0.0,0.0,NaN,NaN,5.6,NaN,NaN,0.0
2004-10-28,5.4,6.2,6.5,5.4,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,1.1,1.3,2.1,0.3,0.0,0.0,0.0,0.0,-1.6,-0.9,-2.0,76.0
2023-11-06,-0.1,0.0,1.0,-1.1,0.0,0.0,0.0,0.0,-2.3,-1.9,-2.8,80.0
2023-11-06,-0.5,-0.4,0.1,-0.7,0.0,0.0,0.0,0.0,-3.1,-2.8,-3.3,83.0


In [11]:
data.isna().sum()

T_CALC           1384
T_HR_AVG         1446
T_MAX            1385
T_MIN            1389
P_CALC            832
SOLARAD           570
SOLARAD_MAX      9757
SOLARAD_MIN      9757
SUR_TEMP          724
SUR_TEMP_MAX     9911
SUR_TEMP_MIN     9911
RH_HR_AVG       48248
dtype: int64

In [12]:
# Check data types
data.dtypes

T_CALC          float64
T_HR_AVG        float64
T_MAX           float64
T_MIN           float64
P_CALC          float64
SOLARAD         float64
SOLARAD_MAX     float64
SOLARAD_MIN     float64
SUR_TEMP        float64
SUR_TEMP_MAX    float64
SUR_TEMP_MIN    float64
RH_HR_AVG       float64
dtype: object

In [13]:
data.shape

(166759, 12)

In [14]:
# forward fill the missing values  
data.ffill(axis = 0, inplace = True) 

In [15]:
data

,T_CALC,T_HR_AVG,T_MAX,T_MIN,P_CALC,SOLARAD,SOLARAD_MAX,SOLARAD_MIN,SUR_TEMP,SUR_TEMP_MAX,SUR_TEMP_MIN,RH_HR_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,8.8,NaN,NaN,0.0
2004-10-27,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6.7,NaN,NaN,0.0
2004-10-28,7.8,7.6,8.0,7.3,0.0,0.0,NaN,NaN,6.1,NaN,NaN,0.0
2004-10-28,6.5,7.0,7.8,6.5,0.0,0.0,NaN,NaN,5.6,NaN,NaN,0.0
2004-10-28,5.4,6.2,6.5,5.4,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,1.1,1.3,2.1,0.3,0.0,0.0,0.0,0.0,-1.6,-0.9,-2.0,76.0
2023-11-06,-0.1,0.0,1.0,-1.1,0.0,0.0,0.0,0.0,-2.3,-1.9,-2.8,80.0
2023-11-06,-0.5,-0.4,0.1,-0.7,0.0,0.0,0.0,0.0,-3.1,-2.8,-3.3,83.0


In [16]:
# drop NaN at the top
data.dropna(inplace = True)

In [17]:
# set target
data['target'] = data['T_HR_AVG']

In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [19]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

with open('./LSTM/models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [20]:
train.shape

(126057, 13)

In [21]:
test.shape

(31515, 13)

In [24]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [30]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, name = 'BiLSTM'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = Bidirectional(LSTM(units=30, activation = 'tanh', input_shape = input_shape, return_sequences=False, name = 'lstm_1'), name = 'bilstm_1')
        self.dense1 = Dense(units=output_shape, activation = 'relu', name = 'dense_1')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x
    
    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [25]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [26]:
X_train.shape

TensorShape([126037, 16, 12])

In [27]:
y_train.shape

TensorShape([126037, 4])

In [37]:
# create model instance
model_name = 'BiLSTM_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_16-4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 4)                 244       
                                                                 
Total params: 10,564
Trainable params: 10,564
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 16s 12ms/step - loss: 0.0016 - mse: 0.0016 - acc: 0.5306 - val_loss: 6.8125e-04 - val_mse: 6.8125e-04 - val_acc: 0.5547
Epoch 2/30
1135/1135 [==============================] - 12s 10ms/step - loss: 6.5089e-04 - mse: 6.5089e-04 - acc: 0.5884 - val_loss: 7.5768e-04 - val_mse: 7.5768e-04 - val_acc: 0.5224
Epoch 3/30
1135/1135 [==============================] - 12s 11ms/step - loss: 5.9310e-04 - mse: 5.9310e-04 - acc: 0.6097 - val_loss: 6.0193e-04 - val_mse: 6.0193e-04 - val_acc: 0.6015
Epoch 4/30
1135/1135 [==============================] - 12s 11ms/step - loss: 5.7056e-04 - mse: 5.7056e-04 - acc: 0.6129 - val_loss: 5.2764e-04 - val_mse: 5.2764e-04 - val_acc: 0.4802
Epoch 5/30
1135/1135 [==============================] - 12s 11ms/step - loss: 5.4763e-04 - mse: 5.4763e-04 - acc: 0.6240 - val_loss: 4.9546e-04 - val_mse: 4.9546e-04 - val_acc: 0.6373
Epoch 6/30
1135/1135 [==============================] - 12s 10ms/step - loss: 5.3440e-04

In [39]:
# save trained model
model.save('./LSTM/models/' + model_name)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_16-4\assets


985/985 [==============================] - 1s 1ms/step


In [40]:
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.00044512811861828965
test set: 0.00047597461054126554


## input length : output length = 24:6

In [41]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [42]:
# create model instance
model_name = 'BiLSTM_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_24-6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 6)                 366       
                                                                 
Total params: 10,686
Trainable params: 10,686
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 12s 9ms/step - loss: 0.0025 - mse: 0.0025 - acc: 0.5023 - val_loss: 9.1943e-04 - val_mse: 9.1943e-04 - val_acc: 0.5922
Epoch 2/30
1135/1135 [==============================] - 9s 8ms/step - loss: 9.5031e-04 - mse: 9.5031e-04 - acc: 0.5522 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5329
Epoch 3/30
1135/1135 [==============================] - 9s 8ms/step - loss: 8.7681e-04 - mse: 8.7681e-04 - acc: 0.5619 - val_loss: 7.7156e-04 - val_mse: 7.7156e-04 - val_acc: 0.5875
Epoch 4/30
1135/1135 [==============================] - 9s 8ms/step - loss: 8.4813e-04 - mse: 8.4813e-04 - acc: 0.5771 - val_loss: 7.7191e-04 - val_mse: 7.7191e-04 - val_acc: 0.5724
Epoch 5/30
1135/1135 [==============================] - 11s 10ms/step - loss: 8.0143e-04 - mse: 8.0143e-04 - acc: 0.5846 - val_loss: 7.8034e-04 - val_mse: 7.8034e-04 - val_acc: 0.5750
Epoch 6/30
1135/1135 [==============================] - 11s 10ms/step - loss: 7.6633e-04 - mse: 7.6633e

In [44]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_24-6\assets


984/984 [==============================] - 2s 2ms/step
mean_squared_error
train set: 0.00079637006400036
test set: 0.0006432738826293087


## input length : output length = 32:8

In [45]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [46]:
# create model instance
model_name = 'BiLSTM_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_32-8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 8)                 488       
                                                                 
Total params: 10,808
Trainable params: 10,808
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 16s 12ms/step - loss: 0.0026 - mse: 0.0026 - acc: 0.4373 - val_loss: 0.0012 - val_mse: 0.0012 - val_acc: 0.4037
Epoch 2/30
1135/1135 [==============================] - 14s 12ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.4994 - val_loss: 0.0013 - val_mse: 0.0013 - val_acc: 0.5161
Epoch 3/30
1135/1135 [==============================] - 13s 12ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5155 - val_loss: 0.0012 - val_mse: 0.0012 - val_acc: 0.3643
Epoch 4/30
1135/1135 [==============================] - 15s 13ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5270 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5546
Epoch 5/30
1135/1135 [==============================] - 17s 15ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5384 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5247
Epoch 6/30
1135/1135 [==============================] - 13s 12ms/step - loss: 0.0010 - mse: 0.0010 - acc: 0.5499 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc:

In [48]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_32-8\assets


984/984 [==============================] - 2s 2ms/step
mean_squared_error
train set: 0.0008957640480050996
test set: 0.0009218894732674655


## input length : output length = 40:10

In [49]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [50]:
# create model instance
model_name = 'BiLSTM_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_40-10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 40, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 10)                610       
                                                                 
Total params: 10,930
Trainable params: 10,930
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)


Epoch 1/30
1135/1135 [==============================] - 20s 15ms/step - loss: 0.0262 - mse: 0.0262 - acc: 0.3476 - val_loss: 0.0028 - val_mse: 0.0028 - val_acc: 0.3716
Epoch 2/30
1135/1135 [==============================] - 15s 13ms/step - loss: 0.0015 - mse: 0.0015 - acc: 0.4740 - val_loss: 0.0018 - val_mse: 0.0018 - val_acc: 0.3668
Epoch 3/30
1135/1135 [==============================] - 15s 13ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.4979 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.4561
Epoch 4/30
1135/1135 [==============================] - 16s 14ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.5044 - val_loss: 0.0015 - val_mse: 0.0015 - val_acc: 0.4685
Epoch 5/30
1135/1135 [==============================] - 16s 14ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5055 - val_loss: 0.0013 - val_mse: 0.0013 - val_acc: 0.4700
Epoch 6/30
1135/1135 [==============================] - 16s 14ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5088 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc:

In [52]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_40-10\assets


984/984 [==============================] - 2s 2ms/step
mean_squared_error
train set: 0.001059352061533607
test set: 0.0011339804367236572
